In [1]:
!pip install -U pip
!pip install -U pandas
!pip install -U sklearn
!pip install -U imbalanced-learn

Requirement already up-to-date: pip in /home/admin/venv-jupyter/lib/python3.7/site-packages (20.1)
Requirement already up-to-date: pandas in /home/admin/venv-jupyter/lib/python3.7/site-packages (1.0.3)
Requirement already up-to-date: sklearn in /home/admin/venv-jupyter/lib/python3.7/site-packages (0.0)
Requirement already up-to-date: imbalanced-learn in /home/admin/venv-jupyter/lib/python3.7/site-packages (0.6.2)


In [2]:
import pandas as pd 
import sys
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

import classifier_tool as tool

In [3]:
label = 'angin_topan'
rep_list = [
    't_lower', 'c_lower', 'tc_lower',
    't_swrem', 'c_swrem', 'tc_swrem',
    't_stem', 'c_stem', 'tc_stem',
    't_swrem_stem', 'c_swrem_stem', 'tc_swrem_stem',
]

f_gold = '/home/admin/text_processing/anotated_data/classifier_gold_standard/preprocessed/sample_banjir_gold_dataset_pre.json'
df_gold = pd.read_json(f_gold)

In [4]:
def test_rep(rep, df, clf, fold=10, feature='tfidf_unigrams'):
    if 'bow_uni_bigrams' == feature:
        print(feature)
        X, vectorizer = tool.construct_bow_uni_and_bigrams(df[rep])
    elif 'bow_unigrams' == feature:
        print(feature)
        X, vectorizer = tool.construct_bow_unigrams(df[rep])
    elif 'bow_bigrams' == feature:
        print(feature)
        X, vectorizer = tool.construct_bow_bigrams(df[rep])
    elif 'tfidf_uni_bigrams' == feature:
        print(feature)
        X, vectorizer = tool.construct_tfidf_uni_and_bigrams(df[rep])
    elif 'tfidf_unigrams' == feature:
        print(feature)
        X, vectorizer = tool.construct_tfidf_unigrams(df[rep])
    elif 'tfidf_bigrams' == feature:
        print(feature)
        X, vectorizer = tool.construct_tfidf_bigrams(df[rep])
    else:
        print('Feature unknown')
    
    y = df['label']
    
    # Imbalance class handling 
    #X, y = SMOTE().fit_sample(X, y)
    #ros = RandomOverSampler(random_state=0)
    #X, y = ros.fit_sample(X, y)
        
    
    report = tool.eval_cv(fold, X, y, clf)
    y_report = [ d['Y'] for d in report ]
    n_report = [ d['N'] for d in report ]
    
    df_y = pd.DataFrame(y_report)
    df_n = pd.DataFrame(n_report)
    df_y.loc['avg'] = df_y.mean()
    df_n.loc['avg'] = df_n.mean()
    df_y_avg = df_y.loc[['avg']]
    df_y_avg = df_y_avg.rename(columns={'precision' : 'y_prec', 'recall':'y_recall', 'f1-score' : 'y_f1', 'support' : 'y_support'})
    
    return df_y_avg, df_y, df_n

In [5]:
clf = SVC()
df_summary = pd.DataFrame()

for rep in rep_list:
    df_y_avg, df_y, df_n = test_rep(rep, df_gold, clf, feature='bow_uni_bigrams')
    df_summary[rep] = df_y_avg.loc['avg']
df_summary

bow_uni_bigrams
bow_uni_bigrams
bow_uni_bigrams
bow_uni_bigrams
bow_uni_bigrams
bow_uni_bigrams
bow_uni_bigrams
bow_uni_bigrams
bow_uni_bigrams
bow_uni_bigrams
bow_uni_bigrams
bow_uni_bigrams


,t_lower,c_lower,tc_lower,t_swrem,c_swrem,tc_swrem,t_stem,c_stem,tc_stem,t_swrem_stem,c_swrem_stem,tc_swrem_stem
y_prec,0.790846,0.843122,0.859303,0.807896,0.857852,0.851038,0.806615,0.860723,0.858433,0.807268,0.861349,0.856565
y_recall,0.608520,0.622190,0.657973,0.616999,0.617693,0.664780,0.568858,0.653658,0.675495,0.611939,0.631893,0.673375
y_f1,0.681456,0.713113,0.741894,0.695857,0.713826,0.743862,0.660666,0.741531,0.750520,0.692677,0.723345,0.751243
y_support,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000


In [6]:
clf = SVC()
df_summary = pd.DataFrame()

for rep in rep_list:
    df_y_avg, df_y, df_n = test_rep(rep, df_gold, clf, feature='bow_unigrams')
    df_summary[rep] = df_y_avg.loc['avg']
df_summary

bow_unigrams
bow_unigrams
bow_unigrams
bow_unigrams
bow_unigrams
bow_unigrams
bow_unigrams
bow_unigrams
bow_unigrams
bow_unigrams
bow_unigrams
bow_unigrams


,t_lower,c_lower,tc_lower,t_swrem,c_swrem,tc_swrem,t_stem,c_stem,tc_stem,t_swrem_stem,c_swrem_stem,tc_swrem_stem
y_prec,0.773080,0.835619,0.848943,0.783778,0.844906,0.862148,0.780144,0.841368,0.860928,0.795338,0.846767,0.863871
y_recall,0.671366,0.630707,0.656459,0.663111,0.618639,0.701563,0.652334,0.662028,0.694449,0.643251,0.670403,0.697474
y_f1,0.714615,0.716427,0.738768,0.715930,0.710694,0.769343,0.708718,0.736434,0.764123,0.706518,0.745006,0.770410
y_support,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000


In [7]:
clf = SVC()
df_summary = pd.DataFrame()

for rep in rep_list:
    df_y_avg, df_y, df_n = test_rep(rep, df_gold, clf, feature='bow_bigrams')
    df_summary[rep] = df_y_avg.loc['avg']
df_summary

bow_bigrams
bow_bigrams
bow_bigrams
bow_bigrams
bow_bigrams
bow_bigrams
bow_bigrams
bow_bigrams
bow_bigrams
bow_bigrams
bow_bigrams
bow_bigrams


,t_lower,c_lower,tc_lower,t_swrem,c_swrem,tc_swrem,t_stem,c_stem,tc_stem,t_swrem_stem,c_swrem_stem,tc_swrem_stem
y_prec,0.935909,0.897024,0.905025,0.941667,0.950000,0.916111,0.920476,0.920455,0.918766,0.924603,0.900714,0.935790
y_recall,0.220777,0.209684,0.275198,0.211530,0.141960,0.196368,0.228402,0.274838,0.363834,0.209089,0.190401,0.269783
y_f1,0.347844,0.335097,0.419086,0.338507,0.242742,0.318616,0.362545,0.420473,0.519672,0.336097,0.303686,0.413466
y_support,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000


In [8]:
clf = SVC()
df_summary = pd.DataFrame()

for rep in rep_list:
    df_y_avg, df_y, df_n = test_rep(rep, df_gold, clf, feature='tfidf_uni_bigrams')
    df_summary[rep] = df_y_avg.loc['avg']
df_summary

tfidf_uni_bigrams
tfidf_uni_bigrams
tfidf_uni_bigrams
tfidf_uni_bigrams
tfidf_uni_bigrams
tfidf_uni_bigrams
tfidf_uni_bigrams
tfidf_uni_bigrams
tfidf_uni_bigrams
tfidf_uni_bigrams
tfidf_uni_bigrams
tfidf_uni_bigrams


,t_lower,c_lower,tc_lower,t_swrem,c_swrem,tc_swrem,t_stem,c_stem,tc_stem,t_swrem_stem,c_swrem_stem,tc_swrem_stem
y_prec,0.932576,0.945584,0.939394,0.947268,0.937103,0.956746,0.926560,0.928198,0.931813,0.914957,0.937695,0.934402
y_recall,0.258294,0.279638,0.319710,0.278278,0.267470,0.298477,0.287453,0.330508,0.372737,0.296518,0.313925,0.363162
y_f1,0.398633,0.427122,0.473611,0.425871,0.412498,0.444218,0.434579,0.481305,0.528686,0.443726,0.463394,0.518051
y_support,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000


In [9]:
clf = SVC()
df_summary = pd.DataFrame()

for rep in rep_list:
    df_y_avg, df_y, df_n = test_rep(rep, df_gold, clf, feature='tfidf_unigrams')
    df_summary[rep] = df_y_avg.loc['avg']
df_summary

tfidf_unigrams
tfidf_unigrams
tfidf_unigrams
tfidf_unigrams
tfidf_unigrams
tfidf_unigrams
tfidf_unigrams
tfidf_unigrams
tfidf_unigrams
tfidf_unigrams
tfidf_unigrams
tfidf_unigrams


,t_lower,c_lower,tc_lower,t_swrem,c_swrem,tc_swrem,t_stem,c_stem,tc_stem,t_swrem_stem,c_swrem_stem,tc_swrem_stem
y_prec,0.899431,0.894549,0.891510,0.898755,0.902158,0.920486,0.879368,0.873099,0.881816,0.874486,0.880352,0.889428
y_recall,0.440095,0.536533,0.550382,0.455687,0.527047,0.563292,0.455257,0.567871,0.589813,0.460335,0.547386,0.576640
y_f1,0.583487,0.667407,0.678048,0.598149,0.662472,0.688565,0.597198,0.684983,0.697922,0.601333,0.672512,0.690105
y_support,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000


In [10]:
clf = SVC()
df_summary = pd.DataFrame()

for rep in rep_list:
    df_y_avg, df_y, df_n = test_rep(rep, df_gold, clf, feature='tfidf_bigrams')
    df_summary[rep] = df_y_avg.loc['avg']
df_summary

tfidf_bigrams
tfidf_bigrams
tfidf_bigrams
tfidf_bigrams
tfidf_bigrams
tfidf_bigrams
tfidf_bigrams
tfidf_bigrams
tfidf_bigrams
tfidf_bigrams
tfidf_bigrams
tfidf_bigrams


,t_lower,c_lower,tc_lower,t_swrem,c_swrem,tc_swrem,t_stem,c_stem,tc_stem,t_swrem_stem,c_swrem_stem,tc_swrem_stem
y_prec,0.966667,1.000000,1.000000,1.000000,1.0,1.0,0.980000,1.00000,1.000000,1.000000,1.000000,1.000000
y_recall,0.114086,0.006364,0.011491,0.095615,0.0,0.0,0.117855,0.00941,0.017123,0.105628,0.002778,0.002857
y_f1,0.199131,0.012334,0.021667,0.171456,0.0,0.0,0.206335,0.01823,0.032569,0.187991,0.005405,0.005556
y_support,31.000000,31.000000,31.000000,31.000000,31.0,31.0,31.000000,31.00000,31.000000,31.000000,31.000000,31.000000
